In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import time

In [ ]:
keyword_list=['Genie','Genie Model','Genie Research','Genie Alphabet','Genie World','Genie World Model','Genie chat','Genie bot','Genie machine learning','Genie AI', 'Genie Google','Genie 2024','Genie Manual','Genie Paper']

In [ ]:
# use selenium to scroll down google search result page to get more than 10 records.
# use beautifulsoup to parse the site page.
# find the title, link and rank of the search results
def google_search_selenium(keyword_list):
    URL_list=pd.DataFrame(columns=('title','link','keyword','rank'))
    for keyword in keyword_list:
        search_word='+'.join(keyword.split())
        URL = f"https://google.com/search?q={search_word}".format(search_word=search_word)
        #selenium scroll down
        driver = webdriver.Chrome()
        driver.get(URL)

        SCROLL_PAUSE_TIME = 2  # Adjust this value as needed
        last_height = driver.execute_script('return document.body.scrollHeight')
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script('return document.body.scrollHeight')
            if new_height == last_height:
                break
            last_height = new_height
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        #parsing the search result page
        rank_num = 1
        for g in soup.find_all('div', class_='yuRUbf'):
            anchors = g.find_all('a')
            if anchors:
                link = anchors[0]['href']
                title = g.find('h3').text
                item = pd.DataFrame({
                    "title": title,
                    "link": link,
                    'keyword':keyword,
                    'rank':rank_num
                    },index=[0])
                URL_list=pd.concat([URL_list,item], axis=0)
            rank_num=rank_num+1
    return URL_list

In [ ]:
# save the results with the timestamp
genie_google_rank=google_search_selenium(keyword_list)
timestamp = datetime.now().strftime('%Y-%m-%d')
genie_google_rank.to_csv(f'genie_google_rank_{timestamp}.csv')